In [ ]:
!pip install pvlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 53.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import pvlib

# ---------------------------
# 1. Cálculo da Carga Elétrica
# ---------------------------
equipamentos = {
    'Equipamento': ['Lâmpada', 'Geladeira', 'Televisão', 'Ventilador', 'Chuveiro', 'Máquina de Lavar', 'Computador', 'Micro-ondas'],
    'Potência (W)': [15, 150, 100, 50, 4000, 500, 200, 1200],
    'Quantidade': [10, 1, 3, 3, 1, 1, 1, 1],
    'Horas de Uso': [5, 10, 4, 8, 0.5, 2, 4, 0.5],
    'Turno de Uso': ['Noite', 'Dia e Noite', 'Noite', 'Noite', 'Dia', 'Dia', 'Dia e Noite', 'Dia']
}

df = pd.DataFrame(equipamentos)
df['Energia (Wh/dia)'] = df['Potência (W)'] * df['Quantidade'] * df['Horas de Uso']
energia_total_diaria = df['Energia (Wh/dia)'].sum()

# Considerando 2 dias de autonomia
energia_total_sistema = energia_total_diaria * 2

# ---------------------------
# 2. Simulação de Geração Fotovoltaica com PVWatts
# ---------------------------
# Definições da localização (exemplo: São Paulo)
localizacao = pvlib.location.Location(
    latitude=-23.5505,
    longitude=-46.6333,
    tz="America/Sao_Paulo"
)

# Parâmetros do módulo FV
potencia_modulo = 300  # Wp por módulo FV
eficiencia_sistema = 0.75  # Perdas do sistema

# Criando uma série de tempo para um dia inteiro (hora a hora)
times = pd.date_range("2024-06-01", periods=24, freq="h", tz=localizacao.tz)

# Obtendo irradiação horária usando modelo de céu limpo
meteo = localizacao.get_clearsky(times)
irradiacao_ghi = meteo["ghi"]  # Global Horizontal Irradiance (W/m²)

# Obtendo posição solar para calcular ângulos
solpos = pvlib.solarposition.get_solarposition(times, localizacao.latitude, localizacao.longitude)

# Convertendo GHI para POA (Plano dos Módulos FV)
# Assumindo inclinação de 20° e azimute voltado para o Norte (0°)
poa_irrad = pvlib.irradiance.get_total_irradiance(
    surface_tilt=20,
    surface_azimuth=0,
    dni=meteo["dni"],
    ghi=meteo["ghi"],
    dhi=meteo["dhi"],
    solar_zenith=solpos["zenith"],
    solar_azimuth=solpos["azimuth"]
)

# Estimando temperatura da célula FV (método SAPM)
temp_ambiente = 25  # Supondo temperatura ambiente de 25°C
wind_speed = 1  # Velocidade do vento em m/s

temp_celula = pvlib.temperature.sapm_cell(
    poa_irrad["poa_global"],
    temp_air=temp_ambiente,
    wind_speed=wind_speed,
    a=-3.47, b=-0.0594, deltaT=3
)

# Usando a irradiação corrigida no plano dos módulos (POA)
pvwatts_potencia = pvlib.pvsystem.pvwatts_dc(
    poa_irrad["poa_global"],  # POA (Plano do Módulo)
    pdc0=potencia_modulo,
    temp_cell=temp_celula,  # Temperatura da célula FV
    gamma_pdc=-0.004  # Coeficiente de temperatura
) * eficiencia_sistema

# Energia diária gerada por um módulo FV (soma das potências horárias)
energia_gerada_modulo = pvwatts_potencia.sum()  # Wh/dia

# Número de módulos FV necessários
num_modulos = np.ceil(energia_total_diaria / energia_gerada_modulo)

# ---------------------------
# 3. Dimensionamento do Banco de Baterias
# ---------------------------
tensao_bateria = 12  # Sistema de 12V
dod = 0.3  # 30% DoD (ou seja, 70% utilizável)
eficiencia_bateria = 0.9  # Eficiência do banco de baterias

# Capacidade necessária considerando DoD
capacidade_bateria_Ah = (energia_total_sistema / (tensao_bateria * eficiencia_bateria * (1 - dod)))

# Número de baterias (considerando baterias de 200Ah)
num_baterias = np.ceil(capacidade_bateria_Ah / 200)

# ---------------------------
# 4. Dimensionamento do Inversor
# ---------------------------
potencia_pico = df['Potência (W)'].sum()
potencia_inversor = potencia_pico
#Incrementar o calculo da potência de surto

# ---------------------------
# 5. Exibir os Resultados
# ---------------------------
print(f"Consumo diário estimado: {energia_total_diaria:.2f} Wh/dia")
print(f"Número de módulos FV necessários: {num_modulos}")
print(f"Capacidade do banco de baterias (com DoD de 30%): {capacidade_bateria_Ah:.2f} Ah")
print(f"Número de baterias de 200Ah necessárias: {num_baterias}")
print(f"Potência mínima do inversor: {potencia_inversor:.2f} W")


Consumo diário estimado: 9050.00 Wh/dia
Número de módulos FV necessários: 8.0
Capacidade do banco de baterias (com DoD de 30%): 2394.18 Ah
Número de baterias de 200Ah necessárias: 12.0
Potência mínima do inversor: 6215.00 W
